Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from model.classes import network
from model import helpers as hlp
from halfmoon import halfmoon_helpers as hmhlp

Create data

In [ ]:
hm = hmhlp.halfMoon(2500, 10, -5, 6)
hmhlp.plotHalfMoon(hm[0], hm[1])

Convert data to dataframes

In [ ]:
df_pts = pd.DataFrame(hm[0], columns=['x', 'y'])
df_moons = pd.DataFrame(hm[1], columns=['in_red'])
df = pd.concat([df_pts, df_moons], axis=1, join='inner')

Split data

In [ ]:
x_cols = ['x', 'y']
y_cols = ['in_red']

df_train, df_val = hlp.split_training_val_data(df)

df_train_x, df_train_y = hlp.split_input_output_data(df_train, x_cols=x_cols, y_cols=y_cols)
df_val_x, df_val_y = hlp.split_input_output_data(df_val, x_cols=x_cols, y_cols=y_cols)

Normalize training and validation input

In [ ]:
df_train_x_norm = hlp.normalize(df_train_x, -15, 25, -1, 1)
df_val_x_norm = hlp.normalize(df_val_x, -15, 25, -1, 1)

Create model instance

In [ ]:
mdl_hm = network.Feed_Forward()
mdl_hm.add_layer(2)
mdl_hm.add_layer(5, activation_function='tanh')
mdl_hm.add_layer(1, activation_function='sigmoid')

Train the model

In [ ]:
history = mdl_hm.train(
    df_train_x_norm,
    df_train_y,
    epochs=50,
    mode='online',
    logging=True,
    adaptive_learning_rate=True,
    weight_decay_factor=0.00005
)

In [ ]:
for l in mdl_hm.layers:
    if l.id > 0: print(l.get_weights())

Validate trained model

In [ ]:
validations = mdl_hm.validate_binary(df_val_x_norm, df_val_y, debug=False)

Plot validated dataset

In [ ]:
arr_val_x = np.array(df_val_x.values.tolist())
arr_val_x_t = arr_val_x.T

preds = []
for v in validations: preds.append(v[0][0])

plt.scatter(arr_val_x_t[0], arr_val_x_t[1], c=preds, cmap='bwr', marker='x')